<a href="https://colab.research.google.com/github/victorcroisfelt/extended-version-random-access-for-ris-aided/blob/main/shannon_nyquist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline 
import matplotlib.pyplot as plt
from matplotlib import rc

import numpy as np

from scipy.constants import speed_of_light
from scipy.fft import fft, fftfreq
from scipy.fftpack import ifft, idct, dct
import scipy.fftpack as spfft
import cvxpy as cvx

# # LaTeX type definitions
# rc('font', **{'family': 'sans serif', 'serif': ['Computer Modern'], 'size': 12})
# rc('text', usetex=True)


In [10]:
def channel_model(bs_gain, bs_pos,
                  ue_gain, ue_pos,
                  ris_size_el, ris_num_els_hor, ris_num_els_ver, ris_configs
                  ):
    """Get downlink channel gains.

    Returns
    -------
    channel_gains_dl : ndarray of shape (num_configs, num_ues)

    """

    # Extract distances and angles
    bs_distance = np.linalg.norm(bs_pos)
    bs_angle = np.arctan2(bs_pos[0], bs_pos[1])

    ue_distances = np.linalg.norm(ue_pos, axis=0)
    ue_angles = np.arctan2(ue_pos[0, :], ue_pos[1, :])

    # Compute DL pahloss of shape (num_ues, )
    num = bs_gain * ue_gain * (ris_size_el * ris_size_el)**2
    den = (4 * np.pi * bs_distance * ue_distances)**2

    const = num/den
    
    pathloss_dl = const * np.cos(bs_angle)**2

    # Compute fundamental frequency 
    fundamental_freq = ris_size_el / wavelength

    # Compute term 1
    term1 = np.sqrt(pathloss_dl) * ris_num_els_ver

    # Compute term 2
    term2 = np.exp(1j * 2 * np.pi * fundamental_freq * ((bs_distance + ue_distances) / ris_size_el))

    # Compute term 3
    term3 = np.exp(-1j  * 2 * np.pi * fundamental_freq * (ris_num_els_hor + 1) / 2 * (np.sin(bs_angle) - np.sin(ue_angles)))

    # Compute term 4
    enumeration_num_els_x = np.arange(1, ris_num_els_hor + 1)

    term4 = np.exp(1j * 2 * np.pi * fundamental_freq * enumeration_num_els_x[:, None, :] * (np.sin(ue_angles)[None, :, :] - np.sin(ris_configs)[:, :, None]))

    return term1, term2, term3, term4

  

    

In [11]:
########################################
# Parameters
########################################

#-----
# Eletromagnetics
#-----


# Signal
carrier_frequency = 3e9
wavelength = speed_of_light / carrier_frequency
wavenumber = 2 * np.pi / wavelength

# Distances
#maximum_distance = 100

# Noise
noise_power = 10 ** (-94.0 / 10)  # mW

#-----
# RIS
#-----

# Number of configurations
ris_num_configs = 1001

# Number of elements
ris_num_els_ver = ris_num_els_hor = 10  # vertical/horizontal

# Size of each element
ris_size_el = wavelength/2

# RIS size along one of the dimensions
ris_size = ris_num_els_ver * ris_size_el

# RIS configurations
ris_configs = np.linspace(0, np.pi/2, ris_num_configs)

# Minimum distance to the RIS
minimum_distance = (2 / wavelength) * ris_size**2

#-----
# BS
#-----

# BS antenna gain
bs_gain = 10**(5/10)

# Position
bs_angle = np.deg2rad([-45.0])
bs_pos = minimum_distance * np.array([np.sin(bs_angle), np.cos(bs_angle)])

#-----
# UE
#-----

# UE antenna gain
ue_gain = 10**(5/10)

# Define specific positions for the UEs
ue_angles_deg = np.array([30.0, 45.0, 60.0])
#ue_angles_deg = np.array([45.0])
ue_angles = np.deg2rad(ue_angles_deg)

# Prepare to save UE positions
ue_pos = np.zeros((3, ue_angles_deg.size))

# Compute UE positions
ue_pos[0, :] = np.sin(ue_angles)
ue_pos[1, :] = np.cos(ue_angles)

ue_pos *= minimum_distance

# Get channel gains for the UEs
term1, term2, term3, term4 = channel_model(bs_gain, bs_pos, ue_gain, ue_pos, ris_size_el, ris_num_els_hor, ris_num_els_ver, ris_configs)

# # Plot channel gain vs number of configurations 
# fig, axes = plt.subplots(2, 1)

# # Create a marker vector
# markers = ['-', '--', '-.', ':']

# # Go through all UEs
# for ue in range(ue_angles_deg.size):

#     label = r'$\theta_{k=' + str(ue) + '} =' + str(np.round(ue_angles_deg[ue], 2)) + '^{\circ}$'

#     axes[0].plot(np.rad2deg(ris_configs), 20*np.log10(np.abs(channel_gains_dl[:, ue])), markers[ue], label=label)
#     axes[1].plot(np.rad2deg(ris_configs), np.rad2deg(np.angle(channel_gains_dl[:, ue])), markers[ue])

# axes[0].set_xlabel(r'config. angle $\theta_s$ in degrees')
# axes[1].set_xlabel(r'config. angle $\theta_s$ in degrees')

# axes[0].set_ylabel('DL channel gain - mag.')
# axes[1].set_ylabel('DL channel gain - phase')

# axes[0].set_xticks(np.arange(0, 100, 10))
# axes[1].set_xticks(np.arange(0, 100, 10))

# axes[0].legend(fontsize='x-small', framealpha=0.5)

# axes[0].grid(color='gray', linestyle=':', linewidth=0.5, alpha=0.5)
# axes[1].grid(color='gray', linestyle=':', linewidth=0.5, alpha=0.5)

# plt.tight_layout()

IndexError: ignored